In [3]:
#!pip install chromadb

In [11]:
# install libraries
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# loading the textfile: no textfile here, just practice codes
loader = DirectoryLoader('filepath', glob='*.txt', loader_cls=TextLoader)
document = loader.load()    # load the document

In [10]:
# chunks object for import
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
test_chunk = text_split.split_documents(document)  # chunks of the document

In [ ]:
test_chunk[0].page_content   # first chunk

##### Creating DB

In [ ]:
persist_directory = 'db'    # --> similar to index on Pinecode but not on cloud

embedding = OpenAIEmbeddings()

vectod_db = Chroma.from_documents(documents=test_chunk, embedding=embedding, persist_directory=persist_directory)
vector_db.persist()                  # persist data in the local disk

In [ ]:
# Making retrieval
retrieval = vector_db.as_retrieval

In [ ]:
# searches the documents used and gives a response.
docs = retrieval.get_relevant_documents('any question based on the txt file used') 
docs 

#### Make a Chain

In [12]:
from langchain.chains import RetrievalQA

In [ ]:
# retrieval with an LLM
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI, chain_type='stuff', retriever=retrieval, return_source_documents=True)

In [ ]:
# how to get refined answer from llm with the aid of retrieval
query = 'any question asked from the text paper'
llm_response = qa_chain.run(query) or qa_chain(query)
doc = vector_db.similarity_search(query)       # cosine similarity score